In [1]:
import pandas as pd
import json
import requests
from time import sleep
import matplotlib.pyplot as plt
import mplleaflet

## Schools 

In [2]:
schools_df = pd.read_csv('./school-directory-and-information/general-information-of-schools.csv')

In [3]:
schools_df.head()

,school_name,url_address,address,postal_code,telephone_no,telephone_no_2,fax_no,fax_no_2,email_address,mrt_desc,...,session_code,mainlevel_code,sap_ind,autonomous_ind,gifted_ind,ip_ind,mothertongue1_code,mothertongue2_code,mothertongue3_code,special_sdp_offered
0,NATIONAL JUNIOR COLLEGE,http://www.nationaljc.moe.edu.sg,37 HILLCREST ROAD,288913,64661144,na,64684535,na,NJC@MOE.EDU.SG,"BOTANIC GARDENS MRT, TAN KAH KEE MRT, SIXTH AV...",...,FULL DAY,MIXED LEVEL,No,No,No,Yes,Chinese,Malay,Tamil,"For the Integrated Programme (Junior High), we..."
1,TEMASEK JUNIOR COLLEGE,http://temasekjc.moe.edu.sg,22 BEDOK SOUTH ROAD,469278,64428066,na,64428762,na,TEMASEK_JC@MOE.EDU.SG,BEDOK MRT,...,FULL DAY,MIXED LEVEL,No,No,No,Yes,Chinese,Malay,Tamil,The TJC 6-Year Integrated Programme provides a...
2,VICTORIA JUNIOR COLLEGE,http://www.victoriajc.moe.edu.sg/,20 MARINE VISTA,449035,64485011,na,64438337,na,victoria_jc@moe.edu.sg,"Nearest MRT Stations: Kembangan, Eunos",...,FULL DAY,JUNIOR COLLEGE,No,No,No,Yes,Chinese,Malay,Tamil,Arts Programme Beyond Borders Programme Highe...
3,EUNOIA JUNIOR COLLEGE,http://www.eunoiajc.moe.edu.sg,53 MOUNT SINAI ROAD,276880,63518388,na,63518399,na,eunoiajc@moe.edu.sg,"Buona Vista MRT Station, Dover MRT Station",...,FULL DAY,JUNIOR COLLEGE,No,No,No,Yes,Chinese,Malay,Tamil,Environmental Literacy Programme Leadership Ad...
4,ANDERSON SERANGOON JUNIOR COLLEGE,www.asrjc.moe.edu.sg,4500 ANG MO KIO AVENUE 6,569843,64596822,na,64598734,na,asrjc@moe.edu.sg,Yio Chu Kang MRT Station,...,FULL DAY,JUNIOR COLLEGE,No,No,No,No,Chinese,Malay,Tamil,Science Research and Enrichment Programme; Civ...


In [4]:
schools_df['mainlevel_code'].unique()

array(['MIXED LEVEL', 'JUNIOR COLLEGE', 'CENTRALISED INSTITUTE',
       'PRIMARY', 'SECONDARY'], dtype=object)

In [5]:
#Keep only the primary, secondary and mixed levels
schools_df = schools_df[(schools_df['mainlevel_code']!='CENTRALISED INSTITUTE') & (schools_df['mainlevel_code']!='JUNIOR COLLEGE')]
schools_df.head()

,school_name,url_address,address,postal_code,telephone_no,telephone_no_2,fax_no,fax_no_2,email_address,mrt_desc,...,session_code,mainlevel_code,sap_ind,autonomous_ind,gifted_ind,ip_ind,mothertongue1_code,mothertongue2_code,mothertongue3_code,special_sdp_offered
0,NATIONAL JUNIOR COLLEGE,http://www.nationaljc.moe.edu.sg,37 HILLCREST ROAD,288913,64661144,na,64684535,na,NJC@MOE.EDU.SG,"BOTANIC GARDENS MRT, TAN KAH KEE MRT, SIXTH AV...",...,FULL DAY,MIXED LEVEL,No,No,No,Yes,Chinese,Malay,Tamil,"For the Integrated Programme (Junior High), we..."
1,TEMASEK JUNIOR COLLEGE,http://temasekjc.moe.edu.sg,22 BEDOK SOUTH ROAD,469278,64428066,na,64428762,na,TEMASEK_JC@MOE.EDU.SG,BEDOK MRT,...,FULL DAY,MIXED LEVEL,No,No,No,Yes,Chinese,Malay,Tamil,The TJC 6-Year Integrated Programme provides a...
12,HWA CHONG INSTITUTION,http://www.hwachong.edu.sg,661 BUKIT TIMAH ROAD,269734,64683955,64683956 / 64665912,67695857,na,admin@hci.edu.sg,TAN KAH KEE MRT,...,SINGLE SESSION,MIXED LEVEL,Yes,No,No,Yes,Chinese,na,na,"MOE Humanities Programme, MOE Art Elective..."
14,BUKIT PANJANG PRIMARY SCHOOL,http://www.bukitpanjangpri.moe.edu.sg,109 CASHEW ROAD,679676,67691912,na,67637462,na,BPPS@MOE.EDU.SG,From Pending LRT station to Choa Chu Kang MRT ...,...,SINGLE SESSION,PRIMARY,No,No,No,No,Chinese,Malay,Tamil,"Learning for Life Programme (LLP) -""i2can danc..."
15,HAIG GIRLS' SCHOOL,http://www.haiggirls.moe.edu.sg,51 KOON SENG ROAD,427072,63440293,na,64474169,na,HAIGGIRLSSCH@MOE.EDU.SG,"Eunos, Dakota",...,SINGLE SESSION,PRIMARY,No,No,No,No,Chinese,Malay,Tamil,LLP - Character and Leadership through Perform...


In [6]:
#Check the number of schools
len(schools_df)

333

In [7]:
search_query = '37 HILLCREST ROAD'
response = requests.get('https://developers.onemap.sg/commonapi/search?searchVal={}&returnGeom=Y&getAddrDetails=Y&pageNum=1'.format(search_query))
response.status_code

200

In [8]:
json.loads(response.text)

{'found': 1,
 'totalNumPages': 1,
 'pageNum': 1,
 'results': [{'SEARCHVAL': 'NATIONAL JUNIOR COLLEGE',
   'BLK_NO': '37',
   'ROAD_NAME': 'HILLCREST ROAD',
   'BUILDING': 'NATIONAL JUNIOR COLLEGE',
   'ADDRESS': '37 HILLCREST ROAD NATIONAL JUNIOR COLLEGE SINGAPORE 288913',
   'POSTAL': '288913',
   'X': '24749.4869699033',
   'Y': '34722.5671630259',
   'LATITUDE': '1.33029291072176',
   'LONGITUDE': '103.804110905425',
   'LONGTITUDE': '103.804110905425'}]}

In [9]:
#Retrieve the name, latiude and longitude
print('Name:',json.loads(response.text)['results'][0]['BUILDING'])
print('Latitude:',json.loads(response.text)['results'][0]['LATITUDE'])
print('Longitude:',json.loads(response.text)['results'][0]['LONGITUDE'])

Name: NATIONAL JUNIOR COLLEGE
Latitude: 1.33029291072176
Longitude: 103.804110905425


In [10]:
#Create lists to append the relevant information to
search_add = []
retrieved_name = []
retrieved_latitude = []
retrieved_longitude =[]

#Iterate through the DataFrame to retrieve the geodata
for num in range(len(schools_df)):
    #Use teh address as the search query and append the search query
    search_query = schools_df.iloc[num]['school_name']
    search_add.append(search_query)
    
    try:
        #enter the search query into the OneMap SG api and append the retrieved geodata
        response = requests.get('https://developers.onemap.sg/commonapi/search?searchVal={}&returnGeom=Y&getAddrDetails=Y&pageNum=1'.format(search_query))
        retrieved_name.append(json.loads(response.text)['results'][0]['BUILDING'])
        retrieved_latitude.append(float(json.loads(response.text)['results'][0]['LATITUDE']))
        retrieved_longitude.append(float(json.loads(response.text)['results'][0]['LONGITUDE']))
    except:
        #Append None if unable to retrieve the geodata
        retrieved_name.append(None)
        retrieved_latitude.append(None)
        retrieved_longitude.append(None)
    
    #Sleep for a minute after 240 entries, as the allowed max request per minute is 250.
    if (num+1)%240==0:
        sleep(60)

sch_latlong = pd.DataFrame({'search_add':search_add,
                            'retrieved_name':retrieved_name,
                            'retrieved_latitude':retrieved_latitude,
                            'retrieved_longitude':retrieved_longitude})

sch_latlong.to_csv('sch_latlong.csv')

In [11]:
sch_latlong.head()

,search_add,retrieved_name,retrieved_latitude,retrieved_longitude
0,NATIONAL JUNIOR COLLEGE,NATIONAL JUNIOR COLLEGE,1.330293,103.804111
1,TEMASEK JUNIOR COLLEGE,TEMASEK JUNIOR COLLEGE,1.318493,103.935567
2,HWA CHONG INSTITUTION,HWA CHONG INSTITUTION,1.325889,103.800747
3,BUKIT PANJANG PRIMARY SCHOOL,BUKIT PANJANG PRIMARY SCHOOL,1.373622,103.769375
4,HAIG GIRLS' SCHOOL,HAIG GIRLS' SCHOOL (U/C),1.311943,103.902903


In [12]:
#Check for unusual values/variance
sch_latlong.describe()

,retrieved_latitude,retrieved_longitude
count,330.000000,330.000000
mean,1.360339,103.839222
std,0.042446,0.071148
min,1.274958,103.687310
25%,1.330365,103.777768
50%,1.353174,103.841460
75%,1.387632,103.897019
max,1.456608,103.962807


In [13]:
sch_latlong[~sch_latlong.apply(lambda x: True if x[0]==x[1] else False,axis=1)]

,search_add,retrieved_name,retrieved_latitude,retrieved_longitude
4,HAIG GIRLS' SCHOOL,HAIG GIRLS' SCHOOL (U/C),1.311943,103.902903
21,MAYFLOWER PRIMARY SCHOOL,MAYFLOWER PRIMARY SCHOOL (U/C),1.376377,103.843253
47,HUAMIN PRIMARY SCHOOL,HUAMIN PRIMARY SCHOOL (U/C),1.426973,103.844240
140,TANJONG KATONG PRIMARY SCHOOL,TANJONG KATONG PRIMARY SCHOOL (U/C),1.305282,103.900128
200,BUKIT PANJANG GOVT. HIGH SCHOOL,None,NaN,NaN
251,ST. ANDREW'S JUNIOR SCHOOL,SAINT ANDREW'S JUNIOR SCHOOL,1.331403,103.865117
252,ST. ANTHONY'S CANOSSIAN PRIMARY SCHOOL,SAINT ANTHONY'S CANOSSIAN PRIMARY SCHOOL,1.334725,103.941235
253,ST. GABRIEL'S PRIMARY SCHOOL,SAINT GABRIEL'S PRIMARY SCHOOL,1.349472,103.862561
254,ST. MARGARET'S PRIMARY SCHOOL,SAINT MARGARET'S PRIMARY SCHOOL,1.303788,103.846251
255,ST. JOSEPH'S INSTITUTION JUNIOR,SAINT JOSEPH'S INSTITUTION JUNIOR,1.317729,103.845612


<div class='alert alert-block alert-warning'>
    Schools with the <strong>U/C</strong> tag are schools undergoing construction and have two occurances on the map (one for current/holding site and one for the site being built). Some schools have no information retrieved.

In [14]:
#Look at the schools with missing information
missing_sch = sch_latlong[pd.isnull(sch_latlong['retrieved_name'])]
missing_sch

,search_add,retrieved_name,retrieved_latitude,retrieved_longitude
200,BUKIT PANJANG GOVT. HIGH SCHOOL,None,NaN,NaN
267,CANOSSA CATHOLIC PRIMARY SCHOOL,None,NaN,NaN
327,"SCHOOL OF THE ARTS, SINGAPORE",None,NaN,NaN


In [15]:
#Correct the missing values (from Google Maps)
sch_latlong.iloc[200]= ['BUKIT PANJANG GOVT. HIGH SCHOOL','BUKIT PANJANG GOVT. HIGH SCHOOL',1.382108,103.738613]
sch_latlong.iloc[267]= ['CANOSSA CATHOLIC PRIMARY SCHOOL','CANOSSA CATHOLIC PRIMARY SCHOOL',1.326600,103.881888]
sch_latlong.iloc[327]= ['SCHOOL OF THE ARTS, SINGAPORE','SCHOOL OF THE ARTS, SINGAPORE',1.299462,103.848707]

In [16]:
#Check which schools are undergoing construction
sch_latlong[sch_latlong['retrieved_name'].str.contains('U/C')]

,search_add,retrieved_name,retrieved_latitude,retrieved_longitude
4,HAIG GIRLS' SCHOOL,HAIG GIRLS' SCHOOL (U/C),1.311943,103.902903
21,MAYFLOWER PRIMARY SCHOOL,MAYFLOWER PRIMARY SCHOOL (U/C),1.376377,103.843253
47,HUAMIN PRIMARY SCHOOL,HUAMIN PRIMARY SCHOOL (U/C),1.426973,103.844240
140,TANJONG KATONG PRIMARY SCHOOL,TANJONG KATONG PRIMARY SCHOOL (U/C),1.305282,103.900128
263,ST. HILDA'S PRIMARY SCHOOL,SAINT HILDA'S PRIMARY SCHOOL (U/C),1.349350,103.937921
276,PEI CHUN PUBLIC SCHOOL,PEI CHUN PUBLIC SCHOOL (U/C),1.337334,103.855632
279,NAN HUA PRIMARY SCHOOL,NAN HUA PRIMARY SCHOOL (U/C),1.319216,103.761108


In [17]:
#Correct the schools with the current location geodata
sch_latlong.iloc[4]= ["HAIG GIRLS' SCHOOL","HAIG GIRLS' SCHOOL",1.382108,103.738613]
sch_latlong.iloc[21]= ['MAYFLOWER PRIMARY SCHOOL','MAYFLOWER PRIMARY SCHOOL',1.376255,103.835788]
sch_latlong.iloc[47]= ['HUAMIN PRIMARY SCHOOL','HUAMIN PRIMARY SCHOOL',1.427953,103.844715]
sch_latlong.iloc[140]= ["TANJONG KATONG PRIMARY SCHOOL","TANJONG KATONG PRIMARY SCHOOL",1.324682,103.938570]
sch_latlong.iloc[263]= ["ST. HILDA'S PRIMARY SCHOOL","ST. HILDA'S PRIMARY SCHOOL",1.349711,103.936117]
sch_latlong.iloc[276]= ['PEI CHUN PUBLIC SCHOOL','PEI CHUN PUBLIC SCHOOL',1.333303,103.858963]
sch_latlong.iloc[279]= ['NAN HUA PRIMARY SCHOOL','NAN HUA PRIMARY SCHOOL',1.319624,103.761507]

In [18]:
#Check for unusual values/variance
sch_latlong.describe()

,retrieved_latitude,retrieved_longitude
count,333.000000,333.000000
mean,1.360382,103.838684
std,0.042324,0.071331
min,1.274958,103.687310
25%,1.330293,103.776108
50%,1.353613,103.840955
75%,1.387061,103.895962
max,1.456608,103.962807


In [19]:
sch_latlong.columns=['search_add','school_name','school_latitude','school_longitude']

In [20]:
sch_latlong.head()

,search_add,school_name,school_latitude,school_longitude
0,NATIONAL JUNIOR COLLEGE,NATIONAL JUNIOR COLLEGE,1.330293,103.804111
1,TEMASEK JUNIOR COLLEGE,TEMASEK JUNIOR COLLEGE,1.318493,103.935567
2,HWA CHONG INSTITUTION,HWA CHONG INSTITUTION,1.325889,103.800747
3,BUKIT PANJANG PRIMARY SCHOOL,BUKIT PANJANG PRIMARY SCHOOL,1.373622,103.769375
4,HAIG GIRLS' SCHOOL,HAIG GIRLS' SCHOOL,1.382108,103.738613


In [21]:
#Save the correct DataFrame
sch_latlong.drop('search_add',axis=1).to_csv('sch_latlong_corrected.csv')

In [22]:
sch_latlong = pd.read_csv('sch_latlong_corrected.csv',index_col=0)

In [23]:
#Plot all the points
plt.figure(figsize=(8,8))
plt.scatter(x=sch_latlong['school_longitude'],y=sch_latlong['school_latitude'],alpha=0.5,edgecolors='k')

mplleaflet.display()

C:\Anaconda3\envs\Flask + env2\lib\site-packages\IPython\core\display.py:689: UserWarning: Consider using IPython.display.IFrame instead
  warnings.warn("Consider using IPython.display.IFrame instead")


In [24]:
sch_latlong.shape,schools_df.shape

((333, 3), (333, 36))

In [25]:
schools_df[['sap_ind','autonomous_ind','gifted_ind','ip_ind']].head()

,sap_ind,autonomous_ind,gifted_ind,ip_ind
0,No,No,No,Yes
1,No,No,No,Yes
12,Yes,No,No,Yes
14,No,No,No,No
15,No,No,No,No


In [26]:
#Concatenate the programme labels to the latlong DataFrame
sch_latlong_prog = pd.concat([sch_latlong,schools_df[['sap_ind','autonomous_ind','gifted_ind','ip_ind']].reset_index(drop=True)],axis=1)

In [27]:
sch_latlong_prog.head()

,school_name,school_latitude,school_longitude,sap_ind,autonomous_ind,gifted_ind,ip_ind
0,NATIONAL JUNIOR COLLEGE,1.330293,103.804111,No,No,No,Yes
1,TEMASEK JUNIOR COLLEGE,1.318493,103.935567,No,No,No,Yes
2,HWA CHONG INSTITUTION,1.325889,103.800747,Yes,No,No,Yes
3,BUKIT PANJANG PRIMARY SCHOOL,1.373622,103.769375,No,No,No,No
4,HAIG GIRLS' SCHOOL,1.382108,103.738613,No,No,No,No


In [28]:
#Save the DataFrame
sch_latlong_prog.to_csv('sch_latlong_prog.csv')

In [29]:
#Save separate files for sap, autonomous schools and shools that offer GEP and IP programmes
sch_latlong_prog[sch_latlong_prog['sap_ind']=='Yes'].to_csv('sap_sch_latlong.csv')
sch_latlong_prog[sch_latlong_prog['autonomous_ind']=='Yes'].to_csv('autonomous_ind_latlong.csv')
sch_latlong_prog[sch_latlong_prog['gifted_ind']=='Yes'].to_csv('gifted_ind_latlong.csv')
sch_latlong_prog[sch_latlong_prog['ip_ind']=='Yes'].to_csv('ip_ind_latlong.csv')